In [10]:
import requests
import json
import pandas as pd
import numpy as np
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 

In [11]:
subscription_key = '937473d5ae6f4a1294115ec2a988584e' 

# Use the endpoint your received from overview section of the Anomaly Detector resource you created
# the endpoint is like https://westus2.api.cognitive.microsoft.com/, different by regions, you need to concat anomalydetector/v1.0/timeseries/entire/detect

# endpoint = 'https://anomaly-detector-resource.cognitiveservices.azure.com/'
endpoint = 'https://westus2.api.cognitive.microsoft.com/anomalydetector/v1.0/timeseries/entire/detect'

In [12]:
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
output_notebook()

Loading BokehJS ...

In [65]:
def detect(endpoint, subscription_key, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

In [66]:
def build_figure(sample_data, sensitivity):
    sample_data['sensitivity'] = sensitivity
    result = detect(endpoint, subscription_key, sample_data)
    columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
          'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
          'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']], 
          'value': [x['value'] for x in sample_data['series']]}
    response = pd.DataFrame(data=columns)
    values = response['value']
    label = response['timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Batch Anomaly Detection ({0} Sensitvity)".format(sensitivity), width=800, height=600)
    for anom in response['isAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['expectedValues'] + response.iloc[index]['upperMargins'] or 
                         values[index] < response.iloc[index]['expectedValues'] - response.iloc[index]['lowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['expectedValues'] + response['upperMargins']
    lowerband = response['expectedValues'] -response['lowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend='Boundary')
    p.line(label, values, legend='Value', color="#2222aa", line_width=1)
    p.line(label, response['expectedValues'], legend='ExpectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

In [67]:
sample_data['sensitivity'] = 95
result = detect(endpoint, subscription_key, sample_data)
columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
      'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
      'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']], 
      'value': [x['value'] for x in sample_data['series']]}
response = pd.DataFrame(data=columns)

In [32]:
response[response['isAnomaly']==True])

14

In [33]:
len(response)

841

In [4]:
sample_data = json.load(open('data/sample_for_anomaly_detection_latest.json'))
sample_data

{'granularity': 'daily',
 'series': [{'timestamp': '2018-03-01T00:00:00Z', 'value': 32858923},
  {'timestamp': '2018-03-02T00:00:00Z', 'value': 29615278},
  {'timestamp': '2018-03-03T00:00:00Z', 'value': 22839355},
  {'timestamp': '2018-03-04T00:00:00Z', 'value': 25948736},
  {'timestamp': '2018-03-05T00:00:00Z', 'value': 34139159},
  {'timestamp': '2018-03-06T00:00:00Z', 'value': 33843985},
  {'timestamp': '2018-03-07T00:00:00Z', 'value': 33637661},
  {'timestamp': '2018-03-08T00:00:00Z', 'value': 32627350},
  {'timestamp': '2018-03-09T00:00:00Z', 'value': 29881076},
  {'timestamp': '2018-03-10T00:00:00Z', 'value': 22681575},
  {'timestamp': '2018-03-11T00:00:00Z', 'value': 24629393},
  {'timestamp': '2018-03-12T00:00:00Z', 'value': 34010679},
  {'timestamp': '2018-03-13T00:00:00Z', 'value': 33893888},
  {'timestamp': '2018-03-14T00:00:00Z', 'value': 33760076},
  {'timestamp': '2018-03-15T00:00:00Z', 'value': 33093515},
  {'timestamp': '2018-03-16T00:00:00Z', 'value': 29945555},
  {'t

In [3]:
sample_data = json.load(open('data/sample_hourly_for_anomaly_detection_entire.json'))
sample_data


{'period': 24,
 'series': [{'timestamp': '2014-12-07T21:00:00Z', 'value': 7290.0979278853},
  {'timestamp': '2014-12-07T22:00:00Z', 'value': 7884.4973233823},
  {'timestamp': '2014-12-07T23:00:00Z', 'value': 7605.0651183363},
  {'timestamp': '2014-12-08T00:00:00Z', 'value': 7418.215575911326},
  {'timestamp': '2014-12-08T01:00:00Z', 'value': 7486.1280819786},
  {'timestamp': '2014-12-08T02:00:00Z', 'value': 7467.915207509225},
  {'timestamp': '2014-12-08T03:00:00Z', 'value': 7470.28855282085},
  {'timestamp': '2014-12-08T04:00:00Z', 'value': 7751.240935484674},
  {'timestamp': '2014-12-08T05:00:00Z', 'value': 7784.169606873174},
  {'timestamp': '2014-12-08T06:00:00Z', 'value': 7681.6739885518255},
  {'timestamp': '2014-12-08T07:00:00Z', 'value': 7822.3363144976},
  {'timestamp': '2014-12-08T08:00:00Z', 'value': 7831.180295563649},
  {'timestamp': '2014-12-08T09:00:00Z', 'value': 7964.88443027655},
  {'timestamp': '2014-12-08T10:00:00Z', 'value': 8237.408888613225},
  {'timestamp': '201

In [68]:
# Hourly Sample
sample_data = json.load(open('data/sample_hourly_for_anomaly_detection_entire.json'))
sample_data['granularity'] = 'hourly'
sample_data['period'] = 24
# 95 sensitivity
build_figure(sample_data,95)

## Latest point anomaly detection
it will track the anomalies after certain points.

In [13]:
endpoint_latest = 'https://westus2.api.cognitive.microsoft.com/anomalydetector/v1.0/timeseries/last/detect'

In [7]:
def detect(endpoint, subscription_key, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)
def build_figure(result, sample_data, sensitivity):
    columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
              'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins']
              , 'value': [x['value'] for x in sample_data['series']], 'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']]}
    response = pd.DataFrame(data=columns)
    values = response['value']
    label = response['timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Anomaly Detection Result ({0} Sensitivity)".format(sensitivity), width=800, height=600)
    for anom in response['isAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['expectedValues'] + response.iloc[index]['upperMargins'] or 
                         values[index] < response.iloc[index]['expectedValues'] - response.iloc[index]['lowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['expectedValues'] + response['upperMargins']
    lowerband = response['expectedValues'] -response['lowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend='Boundary')
    p.line(label, values, legend='value', color="#2222aa", line_width=1)
    p.line(label, response['expectedValues'], legend='expectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

In [15]:
def detect_anomaly(sensitivity):
    sample_data = json.load(open('data/sample_for_anomaly_detection_latest.json'))
    points = sample_data['series']
    skip_point = 35 ## this define the first perdiod of time stamps which will be skipped
    result = {'expectedValues': [None]*len(points), 'upperMargins': [None]*len(points), 
              'lowerMargins': [None]*len(points), 'isNegativeAnomaly': [False]*len(points), 
              'isPositiveAnomaly':[False]*len(points), 'isAnomaly': [False]*len(points)}
    anom_count = 0
    for i in range(skip_point, len(points)+1):
        single_sample_data = {}
        single_sample_data['series'] = points[i-29:i]
        single_sample_data['granularity'] = 'daily'
        single_sample_data['maxAnomalyRatio'] = 0.25
        single_sample_data['sensitivity'] = sensitivity
        single_point = detect(endpoint_latest, subscription_key, single_sample_data)
        if single_point['isAnomaly'] == True:
            anom_count = anom_count + 1

        result['expectedValues'][i-1] = single_point['expectedValue']
        result['upperMargins'][i-1] = single_point['upperMargin']
        result['lowerMargins'][i-1] = single_point['lowerMargin']
        result['isNegativeAnomaly'][i-1] = single_point['isNegativeAnomaly']
        result['isPositiveAnomaly'][i-1] = single_point['isPositiveAnomaly']
        result['isAnomaly'][i-1] = single_point['isAnomaly']
    
    build_figure(result, sample_data, sensitivity)

In [16]:
detect_anomaly(95)